## Análisis de Simulated Annealing para una fuerza de ventas de 5 nodos

Simulated Annhealing es un algoritmo que recibe los siguientes hiperárametros de entrada: 

+ Tmax
+ Tmin 
+ steps 
+ updates 

Los resultados obtenidos pueden verse afectados al variar los valores de tales hiperparámetros. Por otro lado, dependiendo del número de nodos del grafo, estos hiperparámetros también podrían afectar la ruta mínima encontrada por el algoritmo.    

El presente notebook considera la implementación de simulated annhealing para un grafo con 5 nodos. Como primero objetivo se variarán los hiperparámetros y se identificarán aquellos que den mejores resultados. Entendiéndose como mejores resultados; obtener la ruta con menor distancia. Adicionalmente, una vez seleccionados los mejores hiperparámetros, se correrá el algoritmo 10 veces con el fin de realizar un análisis sobre las rutas obtenidas. Particularmente se tiene interés en revisar variaciones en las rutas obtenidas en cada corrida.

Dentro del conjunto de datos que se tienen disponibles, existen varias fuerzas de venta que deben recorrer 5 nodos. Se decidió elegir la fuerza de venta **80993** perteneciente al estado de Nuevo León para llevar a cabo estas pruebas.

In [1]:
!pip install dynaconf
!pip install psycopg2-binary
!pip install simanneal

In [6]:
# Librerías

import pandas as pd
import sys
sys.path.append('../../')
%load_ext autoreload
%autoreload 2

from src import Utileria as ut
from src.models import particle_swarm as ps
from src.models import simulated_annealing as sa

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 1. Búsqueda de mejores hiperparámetros

**1.1 Definición de datos de entrada**
+ Grafo completo de los puntos que debe vistar la fuerza de ventas
+ Hiperámetros con los que correrá el algoritmo

In [8]:
# Se obtiene el dataframe que contiene el grafo de la fuerza de venta a evaluar:
str_Query = 'select id_origen, id_destino, distancia from trabajo.grafos where id_fza_ventas={};'

# En el query se especifica el id_fza_venta del cual se quiere obtener su grafo
df_Grafo = ut.get_data(str_Query.format(80993))
df_Grafo

,id_origen,id_destino,distancia
0,11037,1001402004,0.267770992210818
1,11037,1006681965,0.3250713086280114
2,11037,1020053072,2.3599263518563016
3,11037,1020235635,1.2662382942527688
4,11037,1020402992,0.07674597387253217
5,1001402004,1006681965,0.4982193543875845
6,1001402004,1020053072,2.611292644038669
7,1001402004,1020235635,1.5321611628202725
8,1001402004,1020402992,0.23296305988637042
9,1006681965,1020053072,2.141257356651251


In [15]:
# Se crea el diccionario de hiper-parámetros que se evaluarán

#Default parameters
#Tmax = 25000.0  # Max (starting) temperature
#Tmin = 2.5      # Min (ending) temperature
#steps = 50000   # Number of iterations
#updates = 100   # Number of updates (by default an update prints to stdout)

dict_Hiper_SA = {'tmax': {10000, 25000},
              'tmin': {1,2.5,5},
              'steps': {500,5000},
              'updates': {10,50,100}
              }



**1.2 Gridsearch**

Dentro de la clase Utileria fue definido un método llamado *GridSearch*, el cual recibe como parámetros el grafo de una fuerza de ventas fijo, un diccionario de parámetros, el algoritmo a evaluar y el número de iteraciones que se correrá por cada combinación de hiperámetros. Este método evalúa el algoritmo con todas las combinaciones que se pueden generar a partir del diccionario de parámetros. 

En este caso se considerarán 3 valores de *updates*, 2 valores del *steps*, 2 valores de *Tmax* y 2 de *Tmin* dando lugar a un total de 36 combinaciones. 

Cada combinación de hiperarámetros se correrá 10 veces y como resultado se obtendrá una tabla indicando los Hiperámetros utilizados, las distancias mínima y máxima obtenidas dentro de las 10 corridas; y el número de corridas en que se repitió tal distancia mínima.

In [16]:
%%time

# Se corre el GridSearch para el grafo y los hiperparámetros previamente definidos

df_Resultado = ut.GridSearch(df_Grafo, sa.SimulatedAnnealing, dict_Hiper_SA, 10)

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          6.24    76.00%    20.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          6.22    68.00%    24.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.70    62.00%    26.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          7.80    76.00%    18.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.86    62.00%    24.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.70    68.00%    28.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          7.99    74.00%    30.00%     0:00:00     0:00:00 Temperature 

     1.00000          6.11    70.00%    40.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          8.07   100.00%    40.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000          8.44    80.00%    32.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000          5.77    84.00%    32.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000          5.66    86.00%    32.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000          6.11    84.00%    30.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000          6.23    90.00%    42.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000 

     2.50000          8.48   100.00%    20.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000          5.66    90.00%    30.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000          8.06    90.00%    10.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000          5.60    90.00%    60.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000          5.66    88.00%    34.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000          6.11    92.00%    38.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000          7.55    98.00%    50.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000 

     5.00000          6.34    70.00%    30.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000          8.44   100.00%    40.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000          8.48    90.00%    20.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000          5.85    90.00%    40.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000          7.66   100.00%    40.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000          5.85   100.00%    50.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          6.23    76.00%    32.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000 

     1.00000          5.86    70.00%    40.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.68    60.00%    20.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          6.24    60.00%    20.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.70    60.00%    20.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.87    70.00%    30.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          6.14    50.00%    20.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.66    80.00%    30.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000 

     2.50000          5.85    90.00%    30.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000          7.55    90.00%    50.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000          6.24    90.00%    20.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000          7.91    70.00%    20.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000          5.66    90.00%    40.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000          6.20    60.00%    30.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000          5.66    90.00%    50.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000 

     5.00000          5.81    98.00%    38.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000          7.92    94.00%    36.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000          5.96   100.00%    70.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000          7.99   100.00%    40.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000          8.43    80.00%    40.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000          5.85    90.00%    50.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000          5.66   100.00%    50.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000 

CPU times: user 1min 39s, sys: 5.49 s, total: 1min 44s
Wall time: 1min 48s


In [17]:
# Se muestra el dataframe con los resultados obtenidos de la corrida del GridSearch
pd.options.display.max_colwidth = 100
df_Resultado

,HiperParámetros,Distancia mínima (km),Distancia máxima (km),Frec. rel. dist. min.
0,"{'tmax': 10000, 'tmin': 1.0, 'steps': 5000, 'updates': 100}",5.604,5.604,10/10
1,"{'tmax': 10000, 'tmin': 1.0, 'steps': 5000, 'updates': 10}",5.604,5.604,10/10
2,"{'tmax': 10000, 'tmin': 1.0, 'steps': 5000, 'updates': 50}",5.604,5.604,10/10
3,"{'tmax': 10000, 'tmin': 1.0, 'steps': 500, 'updates': 100}",5.604,5.604,3/10
4,"{'tmax': 10000, 'tmin': 1.0, 'steps': 500, 'updates': 10}",5.604,5.604,4/10
5,"{'tmax': 10000, 'tmin': 1.0, 'steps': 500, 'updates': 50}",5.604,5.604,4/10
6,"{'tmax': 10000, 'tmin': 2.5, 'steps': 5000, 'updates': 100}",5.604,5.604,10/10
7,"{'tmax': 10000, 'tmin': 2.5, 'steps': 5000, 'updates': 10}",5.604,5.604,10/10
8,"{'tmax': 10000, 'tmin': 2.5, 'steps': 5000, 'updates': 50}",5.604,5.604,10/10
9,"{'tmax': 10000, 'tmin': 2.5, 'steps': 500, 'updates': 100}",5.604,5.604,4/10


**1.3 Análisis y Resultados**

En primer lugar es importante mencionar las motivaciones de ciertos de los hiperparámetros elegidos. 

+ Con respecto al número de iteraciones,se escogieron $100$ iteraciones para poder ver la variación entre iteraciones y la estabilidad del algoritmo. El número fue suficiente ya que en ninguna combinación de parámetros hubo variación en la distancia minima. 

+ Sobre el resto de los parámetros, se escogieron valores alrededor e incluyendo los parámetros por omisión del algoritmo. 

Para poder interpretar los resultados mostrados en el dataframe anterior es conveniente recordar que se realizaron 36 pruebas, es decir, se tuvieron 36 combinaciones de hiperparámetros; y cada una de esas combinaciones se corrió 100 veces, dando un total de 360 corridas. En las 100 corridas de cada prueba se registró la distancia mínima obtenida, la distancia máxima y la frecuencia relativa de la distancia mínima. A continuación se enlistan observaciones importantes derivadas de estas pruebas:

+ La distancia mínima y máxima obtenida: $5.6 km$ es la misma para las 36 combinaciones. Esto demuestra que apesar de utilizar los mismos hiperparámetros, el algoritmo da los mismos resultados. 
 
+ Estos resultados pueden ser consecuencia de haber utilizando los parámetros por default, por eso se hizo el siguiente experimento: correr el algoritmo 100 veces solo con 1 paso, updates con valor 2. Tmin con valor 1 y Tmax con valor 2. Fue interesante que la distancia máxima en este caso ya fue más de 8 km.


In [28]:
%%time

# Se corre el GridSearch para el grafo y los hiperparámetros previamente definidos

dict_Hiper_SA = {'tmax': {2}, 'tmin': {1}, 'steps': {1}, 'updates': {2}}


df_Resultado = ut.GridSearch(df_Grafo, sa.SimulatedAnnealing, dict_Hiper_SA, 100)

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          6.11     0.00%     0.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.68   100.00%   100.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          7.90   100.00%   100.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          6.24   100.00%     0.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          7.61   100.00%   100.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.85   100.00%     0.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.85   100.00%     0.00%     0:00:00     0:00:00 Temperature 

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.70   100.00%   100.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.85   100.00%   100.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          8.00   100.00%   100.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.70     0.00%     0.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.70   100.00%     0.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.70     0.00%     0.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.77     0.00%     0.00%     0:00:00     0:00:00 Temperature 

CPU times: user 20.1 s, sys: 216 ms, total: 20.4 s
Wall time: 21.3 s


In [29]:
pd.options.display.max_colwidth = 100
df_Resultado

,HiperParámetros,Distancia mínima (km),Distancia máxima (km),Frec. rel. dist. min.
0,"{'tmax': 2, 'tmin': 1, 'steps': 1, 'updates': 2}",5.604,8.532,1/100


### 2. GridSearch con parametros óptimos

In [23]:
rutas = pd.DataFrame(index=range(100),columns=['km', 'Ruta'])


dict_Hiper_SA = {'tmax': 10000, 'tmin': 1.0, 'steps': 5000, 'updates': 100}
              


In [24]:
rutas

,km,Ruta
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
95,NaN,NaN
96,NaN,NaN
97,NaN,NaN
98,NaN,NaN


In [25]:
for corrida in range(100):
    SA = sa.SimulatedAnnealing(df_Grafo,dict_Hiper_SA)
    SA.Ejecutar()
    
    min_distancia = round(SA.nbr_MejorCosto,3)
    mejor_ruta =  ut.convert(SA.lst_MejorCamino)
    
    rutas.km[corrida] = min_distancia
    rutas.Ruta[corrida] = mejor_ruta
    
    

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          8.14    74.00%    22.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          8.44    76.00%    30.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          8.56    82.00%    30.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          7.99    70.00%    26.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          6.11    72.00%    26.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.81    62.00%    18.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.70    72.00%    30.00%     0:00:00     0:00:00 Temperature 

     1.00000          8.48    82.00%    28.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          8.06    78.00%    26.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.70    72.00%    24.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          6.11    68.00%    34.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.68    80.00%    32.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          6.20    78.00%    32.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          8.14    72.00%    24.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000 

In [26]:
# Primeras 7 rutas obtenidas
rutas.head(7)

,km,Ruta
0,5.604,11037-1020235635-1020053072-1006681965-1001402004-1020402992
1,5.604,11037-1020235635-1020053072-1006681965-1001402004-1020402992
2,5.604,11037-1020235635-1020053072-1006681965-1001402004-1020402992
3,5.604,11037-1020235635-1020053072-1006681965-1001402004-1020402992
4,5.604,11037-1020235635-1020053072-1006681965-1001402004-1020402992
5,5.604,11037-1020235635-1020053072-1006681965-1001402004-1020402992
6,5.604,11037-1020235635-1020053072-1006681965-1001402004-1020402992


In [27]:
# Descripción de las rutas obtenidas

rutas.groupby('Ruta').describe()

km         \
                                                              count unique   
Ruta                                                                         
11037-1020235635-1020053072-1006681965-1001402004-1020402992  100.0    1.0   

                                                                            
                                                                top   freq  
Ruta                                                                        
11037-1020235635-1020053072-1006681965-1001402004-1020402992  5.604  100.0

### Conclusiones:
+ Los resultados del algoritmo son estables.